In [1]:
import json
from time import strftime
from datetime import datetime, timedelta
import urllib.request
#from apscheduler.schedulers.blocking import BlockingScheduler
import time
import requests
import numpy as np
import pymysql

In [22]:
r = requests.get("https://data.epa.gov.tw/api/v1/aqx_p_432?limit=1000&api_key=9be7b239-557b-4c10-9775-78cadfc555e9&format=json", verify=False)
a=json.loads(r.text)
#for i in range(len(a['records'])):#搜尋每個測站
label=a['records'][0]   
print(label['SiteName'],label['County'],label['SiteId'],label['Longitude'],label['Latitude'])

基隆 基隆市 1 121.760056 25.129167


C:\Users\hong5\anaconda3\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.epa.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [23]:
conn = pymysql.Connect(host = '127.0.0.1',
                       port = 3306,
                       user = 'QQAQ',
                       passwd = 'qqaqqqaq',
                       db = 'qqaq',
                       charset='utf8')
H=datetime.now().hour
for i in range(len(a['records'])):#搜尋每個測站
    label=a['records'][i]
    sid=int(label['SiteId'])
    AQI=str(label['AQI'])
    SO2=str(label['SO2'])
    PM25=str(label['PM2.5'])
    PM10=str(label['PM10'])
    CO=str(label['CO'])
    O3=str(label['O3'])
    direct=str(label['WindDirec'])
    speed=str(label['WindSpeed'])
    cur = conn.cursor()
    #sql = "INSERT INTO `station_info`(`id`,`station`, `county`,`longitude`, `latitude`) values(%d,'%s','%s',%f,%f);"%(sid,station,county,longt,laitt)
    #values = (label['SiteId'],label['SiteName'],label['County'],label['Longitude'],label['Latitude'])
    sql = "INSERT INTO `history`(`sid`, `AQI`, `PM10`, `O3`, `CO`, `SO2`, `PM25`, `direct`, `speed`) VALUES( %d,'%s','%s','%s','%s','%s','%s','%s','%s');"%(sid,AQI,PM10,O3,CO,SO2,PM25,direct,speed)
    cur.execute(sql)
    conn.commit()

In [30]:
Y=int(datetime.now().year)
M=int(datetime.now().month)
D=int(datetime.now().day)
H=int(datetime.now().hour)
county = ['基隆','臺北','板橋','新屋','新竹','新竹','國三S156K','臺中','田中','日月潭','古坑','嘉義','國一N250K','永康','高雄','恆春','宜蘭','花蓮','臺東','澎湖','金門','馬祖']
s = requests.get("https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-5702404A-5F4B-4CFE-8162-C11C29E66532&format=JSON&elementName=", verify=False)
b=json.loads(s.text)
for i in range(len(b['records']['location'])):
    city=b['records']['location'][i]['parameter'][0]['parameterValue']
    cout=b['records']['location'][i]['locationName']
    temp=b['records']['location'][i]['weatherElement'][3]['elementValue']#溫度
    humid=b['records']['location'][i]['weatherElement'][4]['elementValue']#濕度
    if float(temp) == -99:
        temp = np.nan
    if float(humid) == -99:
        humid = np.nan
    for j in range(len(county)):
        if cout==county[j]:
            if j == 5:
                city = '新竹市'
            elif j == 12:
                city = '嘉義縣'
            #print(j,city,temp,humid)
            temp=str(temp)
            humid=str(humid)
            cur = conn.cursor()
            #sql = "INSERT INTO `station_info`(`id`,`station`, `county`,`longitude`, `latitude`) values(%d,'%s','%s',%f,%f);"%(sid,station,county,longt,laitt)
            #values = (label['SiteId'],label['SiteName'],label['County'],label['Longitude'],label['Latitude'])
            sql = "UPDATE `history` ,`station_info`SET `temp`='%s',`humid`='%s' WHERE history.sid=station_info.id and station_info.county='%s' and EXTRACT(YEAR FROM history.time)=%d and EXTRACT(MONTH FROM history.time)=%d and EXTRACT(DAY FROM history.time)=%d and EXTRACT(HOUR FROM history.time)=%d;"%(temp,humid,city,Y,M,D,H)
            cur.execute(sql)
            conn.commit()
            #weather.append({'county':city, 'humid':humid, 'temp':temp})

11 嘉義市 14.20 0.59
18 臺東縣 14.30 0.72
1 臺北市 11.30 0.73
3 桃園市 12.20 0.58
12 嘉義縣 12.90 0.61
14 高雄市 15 0.56
6 苗栗縣 11.80 0.59
10 雲林縣 11.90 0.89
0 基隆市 11.70 0.58
7 臺中市 14 0.71
4 新竹縣 12.50 0.53
5 新竹市 12.50 0.53
19 澎湖縣 14.10 0.47
2 新北市 11.50 0.64
16 宜蘭縣 12.60 0.58
15 屏東縣 16.70 0.64
20 金門縣 14 0.27
9 南投縣 8 0.93
17 花蓮縣 13.20 0.87
8 彰化縣 13.70 0.55
21 連江縣 10 0.38
13 臺南市 15 0.53


C:\Users\hong5\anaconda3\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.cwb.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [34]:
county = ['基隆','臺北','板橋','桃園','新竹市東區','竹東','苗栗','臺中','田中','南投','西螺','嘉義','太保','臺南','高雄','屏東','宜蘭','瑞穗','臺東','澎湖','金門','馬祖']
t = requests.get("https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0002-001?Authorization=CWB-5702404A-5F4B-4CFE-8162-C11C29E66532&format=JSON", verify=False)
#api_key可能須定期更新
c=json.loads(t.text)
for i in range(len(c['records']['location'])):
    label=c['records']['location'][i]['locationName']
    rain=c['records']['location'][i]['weatherElement'][1]['elementValue']#雨量
    city=c['records']['location'][i]['parameter'][0]['parameterValue']#城市
    if float(rain)< 0:
        if float(rain) == -998.0:
            rain = 0
        else:
            rain = np.nan
    else:
        rain = 1
    for j in range(len(county)):
        if label==county[j]:
            #print(j+1,city,rain)
            cur = conn.cursor()
            #sql = "INSERT INTO `station_info`(`id`,`station`, `county`,`longitude`, `latitude`) values(%d,'%s','%s',%f,%f);"%(sid,station,county,longt,laitt)
            #values = (label['SiteId'],label['SiteName'],label['County'],label['Longitude'],label['Latitude'])
            sql = "UPDATE `history` ,`station_info`SET `rain`=%d WHERE history.sid=station_info.id and station_info.county='%s' and EXTRACT(YEAR FROM history.time)=%d and EXTRACT(MONTH FROM history.time)=%d and EXTRACT(DAY FROM history.time)=%d and EXTRACT(HOUR FROM history.time)=%d;"%(rain,city,Y,M,D,H)
            cur.execute(sql)
            conn.commit()

C:\Users\hong5\anaconda3\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.cwb.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
